In [0]:
pip install python-dotenv

In [0]:
# Authentication using Service Principal OAuth for Account API
import requests
import json
import os
from dotenv import load_dotenv

# Make sure environment variables are loaded
load_dotenv()

# Get credentials from environment variables
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("SECRET")
ACCOUNT_ID = os.getenv("ACCOUNT_ID")

def get_oauth_token():
    """
    Get an OAuth token using the service principal's client ID and secret.
    This will allow access to account-level APIs.
    """
    # Construct the token endpoint URL correctly for Azure Databricks
    token_url = f"https://accounts.azuredatabricks.net/oidc/accounts/{ACCOUNT_ID}/v1/token"
    
    # Make the token request using HTTP Basic Auth
    # The client_id and client_secret should be passed in the Authorization header
    # using HTTP Basic Auth format (not in the request body)
    response = requests.post(
        token_url,
        auth=(CLIENT_ID, CLIENT_SECRET),  # HTTP Basic Auth
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data="grant_type=client_credentials&scope=all-apis"  # Correct format for the payload
    )
    
    if response.status_code == 200:
        token_data = response.json()
        return token_data["access_token"]
    else:
        print(f"Error getting token: {response.status_code}")
        print(response.text)
        
        # Additional debugging information
        print(f"\nRequest details for debugging:")
        print(f"URL: {token_url}")
        print(f"CLIENT_ID length: {len(CLIENT_ID) if CLIENT_ID else 'None'}")
        print(f"CLIENT_SECRET length: {len(CLIENT_SECRET) if CLIENT_SECRET else 'None'}")
        print(f"ACCOUNT_ID: {ACCOUNT_ID}")
        
        return None

def test_authentication():
    """
    Test authentication by making a simple API call to list budget policies.
    """
    token = get_oauth_token()
    
    if not token:
        print("Failed to obtain OAuth token.")
        return False
    
    # Use the token to make a simple API call
    url = f"https://accounts.azuredatabricks.net/api/2.1/accounts/{ACCOUNT_ID}/budget-policies"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        print("✅ Authentication successful!")
        # Print a sample of the response (first few policies if any)
        policies = response.json().get("policies", [])
        if policies:
            print(f"Found {len(policies)} budget policies.")
            if len(policies) > 0:
                print(f"First policy name: {policies[0].get('policy_name', 'N/A')}")
        else:
            print("No budget policies found, but API call was successful.")
        return True
    else:
        print(f"❌ Authentication failed with status code: {response.status_code}")
        print(response.text)
        return False

# Test authentication and get a token
auth_success = test_authentication()
print(f"Authentication status: {'Successful' if auth_success else 'Failed'}")

# If authentication is successful, store the token for later use
if auth_success:
    # This token can be used for all API calls in this notebook
    TOKEN = get_oauth_token()
    print(f"OAuth Token: {TOKEN[:10]}...{TOKEN[-10:]} (truncated for security)")
    
    # Update the make_api_request function to use OAuth token
    def make_api_request(method, endpoint, data=None, params=None):
        """
        Make a request to the Databricks Account API using OAuth authentication.
        
        Args:
            method (str): HTTP method (GET, POST, PATCH, DELETE)
            endpoint (str): API endpoint path
            data (dict, optional): Request payload
            params (dict, optional): Query parameters
            
        Returns:
            dict: Response from the API
        """
        url = f"https://accounts.azuredatabricks.net{endpoint}"
        headers = {
            "Authorization": f"Bearer {TOKEN}",
            "Content-Type": "application/json"
        }
        
        response = requests.request(
            method=method,
            url=url,
            headers=headers,
            data=json.dumps(data) if data else None,
            params=params
        )
        
        if response.status_code == 200:
            return response.json() if response.content else {}
        else:
            print(f"Error: {response.status_code}")
            print(response.text)
            return None

In [0]:
# Load additional environment variables
# (OAuth authentication is already set up in the previous cell)
from dotenv import load_dotenv
import os
    
# We already loaded .env file in the authentication cell, but we'll do it again to be safe
load_dotenv()

# Keep these variables for other operations as needed
DATABRICKS_INSTANCE = os.getenv("DATABRICKS_INSTANCE")
CLUSTER_ID = os.getenv("CLUSTER_ID")
WAREHOUSE_ID = os.getenv("WAREHOUSE_ID")

# ACCOUNT_ID is already loaded in the authentication cell
# We'll keep it here for reference and backward compatibility
ACCOUNT_ID = os.getenv("ACCOUNT_ID")

# Display environment variables for reference
print(f"DATABRICKS_INSTANCE: {DATABRICKS_INSTANCE}")
print(f"CLUSTER_ID: {CLUSTER_ID}")
print(f"WAREHOUSE_ID: {WAREHOUSE_ID}")
print(f"ACCOUNT_ID: {ACCOUNT_ID}")

### API Request Setup

In [0]:
import requests
import json
from uuid import uuid4

# Note: We're NOT using this function since we defined a better version in the OAuth cell
# This is kept here for reference only
def old_make_api_request(method, endpoint, data=None, params=None):
    """
    DEPRECATED: Use the make_api_request function from the authentication cell instead.
    """
    print("WARNING: Using deprecated API request function. Please use the one defined in the authentication cell.")
    url = f"https://accounts.azuredatabricks.net{endpoint}"
    headers = {
        "Authorization": f"Bearer {TOKEN}",  # This is the old PAT token, not the OAuth token
        "Content-Type": "application/json"
    }
    
    response = requests.request(
        method=method,
        url=url,
        headers=headers,
        data=json.dumps(data) if data else None,
        params=params
    )
    
    if response.status_code == 200:
        return response.json() if response.content else {}
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

# We don't need to redefine make_api_request here - we'll use the one from the OAuth cell
# The function is already defined with our OAuth token in cell #2

### List Budget Policies

In [0]:
def list_budget_policies(page_size=100, page_token=None, filter_by=None, sort_spec=None):
    """
    List all budget policies.
    
    Args:
        page_size (int, optional): Maximum number of policies to return (max 1000)
        page_token (str, optional): Token for pagination
        filter_by (dict, optional): Filter criteria
        sort_spec (dict, optional): Sort specification
        
    Returns:
        dict: List of budget policies
    """
    endpoint = f"/api/2.1/accounts/{ACCOUNT_ID}/budget-policies"
    params = {}
    
    if page_size:
        params["page_size"] = page_size
    if page_token:
        params["page_token"] = page_token
    if filter_by:
        params["filter_by"] = filter_by
    if sort_spec:
        params["sort_spec"] = sort_spec
    
    # This will use the make_api_request function defined in cell #2 with OAuth token
    return make_api_request("GET", endpoint, params=params)

# Example: List all budget policies
policies = list_budget_policies()
print(json.dumps(policies, indent=2))

### Create Budget Policies

In [0]:
def create_budget_policy(policy_name, binding_workspace_ids=None, custom_tags=None):
    """
    Create a new budget policy.
    
    Args:
        policy_name (str): Name of the policy (1-127 chars)
        binding_workspace_ids (list, optional): List of workspace IDs to bind the policy to
        custom_tags (list, optional): List of custom tags (max 20)
        
    Returns:
        dict: Created budget policy
    """
    endpoint = f"/api/2.1/accounts/{ACCOUNT_ID}/budget-policies"
    
    policy = {
        "policy_name": policy_name
    }
    
    if binding_workspace_ids:
        policy["binding_workspace_ids"] = binding_workspace_ids
    
    if custom_tags:
        policy["custom_tags"] = custom_tags
    
    data = {
        "policy": policy,
        "request_id": str(uuid4())  # Generate a random UUID for idempotent request
    }
    
    return make_api_request("POST", endpoint, data=data)

# Example: Create a new budget policy
# Uncomment and modify as needed
new_policy = create_budget_policy(
    policy_name="Dev Team Budget",
    binding_workspace_ids=[12345, 67890],  # Replace with actual workspace IDs
    custom_tags=[
        {"key": "department", "value": "engineering"},
        {"key": "project", "value": "data-platform"}
    ]
)
print(json.dumps(new_policy, indent=2))

### Get budget by ID

In [0]:
def get_budget_policy(policy_id):
    """
    Get a budget policy by ID.
    
    Args:
        policy_id (str): ID of the policy
        
    Returns:
        dict: Budget policy details
    """
    endpoint = f"/api/2.1/accounts/{ACCOUNT_ID}/budget-policies/{policy_id}"
    return make_api_request("GET", endpoint)

# Example: Get a budget policy by ID
# Uncomment and modify as needed
policy_id = "dee573ea-fc87-3782-a2ca-9bd87c7d8bbf"  # Replace with actual policy ID
policy = get_budget_policy(policy_id)
print(json.dumps(policy, indent=2))

### Update budget policy

In [0]:
def update_budget_policy(policy_id, policy_name=None, binding_workspace_ids=None, custom_tags=None):
    """
    Update a budget policy.
    
    Args:
        policy_id (str): ID of the policy to update
        policy_name (str, optional): New name for the policy
        binding_workspace_ids (list, optional): New list of workspace IDs
        custom_tags (list, optional): New list of custom tags
        
    Returns:
        dict: Updated budget policy
    """
    endpoint = f"/api/2.1/accounts/{ACCOUNT_ID}/budget-policies/{policy_id}"
    
    data = {
        "policy_id": policy_id
    }
    
    if policy_name:
        data["policy_name"] = policy_name
    
    if binding_workspace_ids is not None:  # Allow empty list to clear bindings
        data["binding_workspace_ids"] = binding_workspace_ids
    
    if custom_tags is not None:  # Allow empty list to clear tags
        data["custom_tags"] = custom_tags
    
    return make_api_request("PATCH", endpoint, data=data)

# Example: Update a budget policy
# Uncomment and modify as needed
policy_id = "dee573ea-fc87-3782-a2ca-9bd87c7d8bbf"  # Replace with actual policy ID
updated_policy = update_budget_policy(
    policy_id=policy_id,
    policy_name="Updated Dev Team Budget",
    binding_workspace_ids=[],  # Add a new workspace ID
    custom_tags=[
        {"key": "department", "value": "engineering"},
        {"key": "project", "value": "data-platform"},
        {"key": "status", "value": "active"}
    ]
)
print(json.dumps(updated_policy, indent=2))

### Delete budget ID

In [0]:
def delete_budget_policy(policy_id):
    """
    Delete a budget policy.
    
    Args:
        policy_id (str): ID of the policy to delete
        
    Returns:
        dict: Empty response if successful
    """
    endpoint = f"/api/2.1/accounts/{ACCOUNT_ID}/budget-policies/{policy_id}"
    return make_api_request("DELETE", endpoint)

# Example: Delete a budget policy
# Uncomment and modify as needed
policy_id = "dee573ea-fc87-3782-a2ca-9bd87c7d8bbf"  # Replace with actual policy ID
result = delete_budget_policy(policy_id)
if result is not None:
    print(f"Policy {policy_id} deleted successfully.")
else:
    print(f"Failed to delete policy {policy_id}.")

### Prettify Policy

In [0]:
def prettify_policy(policy):
    """
    Format a policy for better readability.
    
    Args:
        policy (dict): Budget policy
        
    Returns:
        str: Formatted policy string
    """
    output = []
    output.append(f"Policy ID: {policy.get('policy_id', 'N/A')}")
    output.append(f"Name: {policy.get('policy_name', 'N/A')}")
    
    binding_workspace_ids = policy.get('binding_workspace_ids', [])
    if binding_workspace_ids:
        output.append(f"Binding Workspace IDs: {', '.join(map(str, binding_workspace_ids))}")
    else:
        output.append("Binding Workspace IDs: None (applies to all workspaces)")
    
    custom_tags = policy.get('custom_tags', [])
    if custom_tags:
        output.append("Custom Tags:")
        for tag in custom_tags:
            output.append(f"  - {tag.get('key', 'N/A')}: {tag.get('value', 'N/A')}")
    else:
        output.append("Custom Tags: None")
    
    return "\n".join(output)

def display_policies(policies_response):
    """
    Display a list of policies in a more readable format.
    
    Args:
        policies_response (dict): Response from list_budget_policies
    """
    if not policies_response:
        print("No response or error occurred.")
        return
    
    policies = policies_response.get('policies', [])
    next_page_token = policies_response.get('next_page_token')
    previous_page_token = policies_response.get('previous_page_token')
    
    print(f"Total policies: {len(policies)}")
    
    if next_page_token:
        print(f"Next page token: {next_page_token}")
    
    if previous_page_token:
        print(f"Previous page token: {previous_page_token}")
    
    print("\nPolicies:")
    for i, policy in enumerate(policies, 1):
        print(f"\n{i}. " + prettify_policy(policy))
        print("-" * 50)

# Example: Display policies in a readable format
# Uncomment to run
all_policies = list_budget_policies()
display_policies(all_policies)

### End to end example

In [0]:
def run_complete_example():
    """
    Complete example of budget policy management workflow.
    """
    # Step 1: Create a new policy
    print("Creating a new budget policy...")
    new_policy = create_budget_policy(
        policy_name="Example Budget Policy",
        binding_workspace_ids=[12345],  # Replace with actual workspace ID
        custom_tags=[
            {"key": "department", "value": "data_science"},
            {"key": "environment", "value": "dev"}
        ]
    )
    
    if not new_policy:
        print("Failed to create a policy. Exiting example.")
        return
    
    policy_id = new_policy["policy_id"]
    print(f"Created policy with ID: {policy_id}")
    print(prettify_policy(new_policy))
    
    # Step 2: Get the policy details
    print("\nFetching the policy details...")
    policy = get_budget_policy(policy_id)
    print(prettify_policy(policy))
    
    # Step 3: Update the policy
    print("\nUpdating the policy...")
    updated_policy = update_budget_policy(
        policy_id=policy_id,
        policy_name="Updated Example Policy",
        binding_workspace_ids=[12345, 67890],  # Replace with actual workspace IDs
        custom_tags=[
            {"key": "department", "value": "data_science"},
            {"key": "environment", "value": "staging"},
            {"key": "status", "value": "active"}
        ]
    )
    print(prettify_policy(updated_policy))
    
    # Step 4: List all policies
    print("\nListing all policies...")
    all_policies = list_budget_policies()
    display_policies(all_policies)
    
    # Step 5: Delete the policy
    print(f"\nDeleting policy {policy_id}...")
    confirm = input(f"Delete policy {policy_id}? (y/n): ")
    
    if confirm.lower() == 'y':
        result = delete_budget_policy(policy_id)
        if result is not None:
            print(f"Policy {policy_id} deleted successfully.")
            
            # Verify deletion
            print("\nVerifying deletion by listing all policies again...")
            all_policies_after_deletion = list_budget_policies()
            display_policies(all_policies_after_deletion)
        else:
            print(f"Failed to delete policy {policy_id}.")
    else:
        print("Policy deletion cancelled.")

# Uncomment to run the complete example
run_complete_example()

### Validation function

In [0]:
def validate_policy_name(name):
    """
    Validate policy name according to API requirements.
    
    Args:
        name (str): Policy name to validate
        
    Returns:
        bool: True if valid, False otherwise
    """
    if not name:
        print("Error: Policy name cannot be empty.")
        return False
    
    if len(name) > 127:
        print("Error: Policy name must be at most 127 characters.")
        return False
    
    if name.startswith("databricks:default-policy"):
        print("Error: Policy name cannot start with reserved keyword 'databricks:default-policy'.")
        return False
    
    # Check for ISO 8859-1 (latin1) character set
    try:
        name.encode('latin1')
    except UnicodeEncodeError:
        print("Error: Policy name can only contain characters from the ISO 8859-1 (latin1) set.")
        return False
    
    return True

def validate_custom_tags(tags):
    """
    Validate custom tags according to API requirements.
    
    Args:
        tags (list): List of tag dictionaries to validate
        
    Returns:
        bool: True if valid, False otherwise
    """
    if len(tags) > 20:
        print("Error: At most 20 tags are allowed per policy.")
        return False
    
    for tag in tags:
        if 'key' not in tag or 'value' not in tag:
            print("Error: Each tag must have both 'key' and 'value' fields.")
            return False
    
    return True

### CLI for Budget Policy Management

In [0]:
def budget_policy_cli():
    """
    Interactive command-line interface for managing budget policies.
    """
    while True:
        print("\n=== Budget Policy Management CLI ===")
        print("1. List all policies")
        print("2. Create a new policy")
        print("3. Get a policy by ID")
        print("4. Update a policy")
        print("5. Delete a policy")
        print("6. Exit")
        
        choice = input("\nEnter your choice (1-6): ")
        
        if choice == '1':
            # List policies with optional pagination
            page_size = input("Enter page size (press Enter for default 100): ")
            page_size = int(page_size) if page_size else 100
            
            page_token = input("Enter page token (press Enter for none): ")
            page_token = page_token if page_token else None
            
            policies = list_budget_policies(page_size=page_size, page_token=page_token)
            display_policies(policies)
        
        elif choice == '2':
            # Create new policy
            policy_name = input("Enter policy name: ")
            
            if not validate_policy_name(policy_name):
                continue
            
            workspace_ids_str = input("Enter workspace IDs (comma-separated, press Enter for none): ")
            workspace_ids = [int(id.strip()) for id in workspace_ids_str.split(',') if id.strip()] if workspace_ids_str else []
            
            tags_count = input("Enter number of custom tags (press Enter for none): ")
            custom_tags = []
            
            if tags_count and int(tags_count) > 0:
                for i in range(int(tags_count)):
                    key = input(f"Enter tag {i+1} key: ")
                    value = input(f"Enter tag {i+1} value: ")
                    custom_tags.append({"key": key, "value": value})
            
            if custom_tags and not validate_custom_tags(custom_tags):
                continue
            
            new_policy = create_budget_policy(
                policy_name=policy_name,
                binding_workspace_ids=workspace_ids,
                custom_tags=custom_tags
            )
            
            if new_policy:
                print("\nPolicy created successfully:")
                print(prettify_policy(new_policy))
            else:
                print("\nFailed to create policy.")
        
        elif choice == '3':
            # Get policy by ID
            policy_id = input("Enter policy ID: ")
            policy = get_budget_policy(policy_id)
            
            if policy:
                print("\nPolicy details:")
                print(prettify_policy(policy))
            else:
                print(f"\nPolicy with ID {policy_id} not found or error occurred.")
        
        elif choice == '4':
            # Update existing policy
            policy_id = input("Enter policy ID to update: ")
            
            # Get current policy to show current values
            current_policy = get_budget_policy(policy_id)
            
            if not current_policy:
                print(f"\nPolicy with ID {policy_id} not found or error occurred.")
                continue
            
            print("\nCurrent policy details:")
            print(prettify_policy(current_policy))
            
            # Get updated values
            policy_name = input(f"\nEnter new policy name (press Enter to keep '{current_policy.get('policy_name')}'): ")
            if policy_name and not validate_policy_name(policy_name):
                continue
            policy_name = policy_name if policy_name else current_policy.get('policy_name')
            
            workspace_ids_str = input("Enter new workspace IDs (comma-separated, press Enter to keep current): ")
            if workspace_ids_str:
                workspace_ids = [int(id.strip()) for id in workspace_ids_str.split(',') if id.strip()]
            else:
                workspace_ids = current_policy.get('binding_workspace_ids')
            
            update_tags = input("Update custom tags? (y/n): ").lower() == 'y'
            
            if update_tags:
                tags_count = input("Enter number of custom tags: ")
                custom_tags = []
                
                if tags_count and int(tags_count) > 0:
                    for i in range(int(tags_count)):
                        key = input(f"Enter tag {i+1} key: ")
                        value = input(f"Enter tag {i+1} value: ")
                        custom_tags.append({"key": key, "value": value})
                
                if custom_tags and not validate_custom_tags(custom_tags):
                    continue
            else:
                custom_tags = current_policy.get('custom_tags')
            
            updated_policy = update_budget_policy(
                policy_id=policy_id,
                policy_name=policy_name,
                binding_workspace_ids=workspace_ids,
                custom_tags=custom_tags
            )
            
            if updated_policy:
                print("\nPolicy updated successfully:")
                print(prettify_policy(updated_policy))
            else:
                print("\nFailed to update policy.")
        
        elif choice == '5':
            # Delete policy
            policy_id = input("Enter policy ID to delete: ")
            
            # Confirm deletion
            confirm = input(f"Are you sure you want to delete policy {policy_id}? (y/n): ").lower()
            
            if confirm == 'y':
                result = delete_budget_policy(policy_id)
                
                if result is not None:
                    print(f"\nPolicy {policy_id} deleted successfully.")
                else:
                    print(f"\nFailed to delete policy {policy_id}.")
            else:
                print("\nDeletion cancelled.")
        
        elif choice == '6':
            print("\nExiting Budget Policy Management CLI. Goodbye!")
            break
        
        else:
            print("\nInvalid choice. Please enter a number between 1 and 6.")

# Uncomment to run the interactive CLI
budget_policy_cli()